In [1]:
from sklearn.externals.joblib import Memory
from sklearn.datasets import load_svmlight_file

X_train, y_train = load_svmlight_file("a9a.txt")[0], load_svmlight_file("a9a.txt")[1]
X_test, y_test = load_svmlight_file("a9a.t.txt")[0], load_svmlight_file("a9a.t.txt")[1]


import numpy as np
X_train = X_train.dot(np.eye(X_train.shape[1]))
X_test = X_test.dot(np.eye(X_test.shape[1]))

X_train = np.hstack((X_train,np.ones((X_train.shape[0],1)))) #把所有x的属性加上一个1，用于和w14（b）相乘

X_test = np.hstack((X_test,np.ones((X_test.shape[0],1)))) #把所有x的属性加上一个0，（补齐数据文件的全零特征）
X_test = np.hstack((X_test,np.ones((X_test.shape[0],1)))) #把所有x的属性加上一个1，用于和w14（b）相乘

y_train = y_train.reshape((y_train.shape[0],1))
y_test = y_test.reshape((y_test.shape[0],1))

w_sgd = np.zeros((X_train.shape[1],1)) #将SGD模型参数w初始化为全零
w_nag = np.zeros((X_train.shape[1],1)) #将NAG模型参数w初始化为全零
w_rmsp = np.zeros((X_train.shape[1],1)) #将RMSProp模型参数w初始化为全零
w_adaD = np.zeros((X_train.shape[1],1)) #将AdaDelta模型参数w初始化为全零
w_adam = np.zeros((X_train.shape[1],1)) #将Adam模型参数w初始化为全零

n = 10000 #设置迭代次数

LSGD = np.zeros((n)) #初始化用于保存LSGD的值随迭代次数变化的数组
LNAG = np.zeros((n)) #初始化用于保存LNAG的值随迭代次数变化的数组
LRMSProp = np.zeros((n)) #初始化用于保存LRMSProp的值随迭代次数变化的数组
LAdaDelta = np.zeros((n)) #初始化用于保存LAdaDelta的值随迭代次数变化的数组
LAdam = np.zeros((n)) #初始化用于保存LAdam的值随迭代次数变化的数组

def Lfun(w,X,y):#定义Loss函数
    m = y.shape[0]
    l = 1-(X.dot(w))*y
    for i in range (m):
        if l[i] < 0:
            l[i] = 0
    return l.sum()+0.5*np.sum(w*w)

def DER_SGD(w,X,y):#定义随机梯度下降的梯度函数
    m = 100 #从数据集中随机取样
    index = np.random.randint(0,X.shape[0],m)
    X_sample = np.zeros((m,X.shape[1]))
    y_sample = np.zeros((m,y.shape[1]))
 
    for i in range (m):
        X_sample[m-1] = X[index[m-1]]
        y_sample[m-1] = y[index[m-1]]
        
    j = (X_sample.dot(w))*y_sample
    o = np.zeros((m,1))
    for i in range (m):
        if j[i] < 1:
            o[i] = y_sample[i]
    return -((X_sample.T).dot(o))

def cRate(w,X,y):#定义用于计算预测正确率的函数
    m = y.shape[0]
    j = (X.dot(w))*y
    c = 0
    for i in range (m):
        if j[i] > 0:
            c = c + 1
    return c/m

def SGD(w,lnrt):
    lnrt = lnrt
    for i in range (n):#迭代若干次，更新模型参数w
        g = (DER_SGD(w,X_train,y_train))
        w -= lnrt*g
        LSGD[i] = Lfun(w,X_test,y_test)

def NAG(w,lnrt,u):
    V = 0
    u = u
    lnrt = lnrt
    for i in range (n):#迭代若干次，更新模型参数w
        g = (DER_SGD(w,X_train,y_train))
        V = u*V + lnrt*g
        w -= V
        LNAG[i] = Lfun(w,X_test,y_test)
    
def RMSProp(w,lnrt,u,e):
    G = 0
    u = u
    e = e
    lnrt = lnrt
    for i in range (n):#迭代若干次，更新模型参数w
        g = (DER_SGD(w,X_train,y_train))
        G = u*G + (1-u)*g*g
        w -=  (lnrt/((G+e)**0.5))*g
        LRMSProp[i] = Lfun(w,X_test,y_test)
        
def AdaDelta(w,u,e):
    G = 0
    u = u
    delta = 0
    e = e
    for i in range (n):#迭代若干次，更新模型参数w
        g = (DER_SGD(w,X_train,y_train))
        G = u*G + (1-u)*(g*g)
        wdelta = -(((delta+e)**0.5)/((G+e)**0.5)) * g
        w += wdelta
        delta = u*delta + (1-u)*(wdelta*wdelta)
        LAdaDelta[i] = Lfun(w,X_test,y_test)
        
def Adam(w,lnrt,u,e,B):
    G = 0
    u = u
    B = B
    m = 0
    e = e
    lnrt = lnrt
    for i in range (n):#迭代若干次，更新模型参数w
        g = (DER_SGD(w,X_train,y_train))
        m = B*m + (1-B)*g
        G = u*G + (1-u)*(g*g)
        w -= (lnrt*((1-u**(i+1))**0.5)*m)/((1-B**(i+1))*((G+e)**0.5))
        LAdam[i] = Lfun(w,X_test,y_test)

SGD(w_sgd,0.001)
NAG(w_nag,0.0001,0.9)
RMSProp(w_rmsp,0.001,0.9,1e-8)
AdaDelta(w_adaD,0.8,1e-6)
Adam(w_adam,0.001,0.999,1e-8,0.9)

import matplotlib.pyplot as plt
#绘制总图
x = np.arange(0,n,1)
plt.rcParams['figure.figsize']=(20,10)
plt.plot(x, LSGD,  'black',label = 'LSGD')
plt.plot(x, LNAG,  'r',label = 'LNAG')
plt.plot(x, LRMSProp,  'b',label = 'LARMSProp')
plt.plot(x, LAdaDelta,  'y',label = 'LAdaDelta')
plt.plot(x, LAdam,  'g',label = 'LAdam')
plt.legend(loc='upper right')
plt.xlabel('Times of iteration')
plt.ylabel('Loss')
plt.show()


FileNotFoundError: [Errno 2] No such file or directory: 'a9a.txt'